## Importing Libraries

In [1]:
# !pip install mediapipe-model-maker


In [2]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import image_classifier

import matplotlib.pyplot as plt

## Loading Dataset

In [10]:
#save final weights to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
zip_file_name = "/content/drive/MyDrive/Colab\ Notebooks/Dataset/dataset" #@param {type:"string"}
!unzip {zip_file_name}.zip

Archive:  /content/drive/MyDrive/Colab Notebooks/Dataset/dataset.zip
   creating: dataset/
   creating: dataset/apple/
  inflating: dataset/apple/Image_1.jpg  
  inflating: dataset/apple/Image_10.jpg  
  inflating: dataset/apple/Image_16.jpg  
  inflating: dataset/apple/Image_17.jpg  
  inflating: dataset/apple/Image_18.jpg  
  inflating: dataset/apple/Image_19.jpg  
  inflating: dataset/apple/Image_2.jpg  
  inflating: dataset/apple/Image_20.jpg  
  inflating: dataset/apple/Image_21.jpg  
  inflating: dataset/apple/Image_23.jpg  
  inflating: dataset/apple/Image_24.jpg  
  inflating: dataset/apple/Image_25.jpg  
  inflating: dataset/apple/Image_26.jpg  
  inflating: dataset/apple/Image_27.jpg  
  inflating: dataset/apple/Image_28.jpg  
  inflating: dataset/apple/Image_3.jpg  
  inflating: dataset/apple/Image_31.jpg  
  inflating: dataset/apple/Image_32.jpg  
  inflating: dataset/apple/Image_33.jpg  
  inflating: dataset/apple/Image_34.jpg  
  inflating: dataset/apple/Image_35.png  
  

In [15]:
image_path = '/content/dataset'

In [16]:
print(image_path)
labels = []
for i in os.listdir(image_path):
  if os.path.isdir(os.path.join(image_path, i)):
    labels.append(i)
print(labels)

/content/dataset
['onion', 'apple', 'lemon', 'potato', 'watermelon', 'banana']


In [17]:
NUM_EXAMPLES = 5

for label in labels:
  label_dir = os.path.join(image_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

In [18]:
data = image_classifier.Dataset.from_folder(image_path)
train_data, remaining_data = data.split(0.9)
test_data, validation_data = remaining_data.split(0.5)

## Model Training

In [19]:
spec = image_classifier.SupportedModels.EFFICIENTNET_LITE2
hparams = image_classifier.HParams(export_dir="exported_model",epochs=50)
options = image_classifier.ImageClassifierOptions(supported_model=spec, hparams=hparams)

In [20]:
model = image_classifier.ImageClassifier.create(
    train_data = train_data,
    validation_data = validation_data,
    options=options,
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              4869168   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 6)                 7686      
                                                                 
Total params: 4876854 (18.60 MB)
Trainable params: 7686 (30.02 KB)
Non-trainable params: 4869168 (18.57 MB)
_________________________________________________________________
None


Instructions for updating:
Use `tf.image.resize(...method=ResizeMethod.BICUBIC...)` instead.


Epoch 1/50
216/216 [==============================] - 82s 354ms/step - loss: 1.8510 - accuracy: 0.2315 - val_loss: 1.6599 - val_accuracy: 0.3200
Epoch 2/50
216/216 [==============================] - 78s 362ms/step - loss: 1.6906 - accuracy: 0.3287 - val_loss: 1.5026 - val_accuracy: 0.4400
Epoch 3/50
216/216 [==============================] - 97s 449ms/step - loss: 1.4577 - accuracy: 0.5463 - val_loss: 1.3271 - val_accuracy: 0.5200
Epoch 4/50
216/216 [==============================] - 74s 341ms/step - loss: 1.2743 - accuracy: 0.6875 - val_loss: 1.1764 - val_accuracy: 0.7200
Epoch 5/50
216/216 [==============================] - 75s 349ms/step - loss: 1.1061 - accuracy: 0.7986 - val_loss: 1.0519 - val_accuracy: 0.8800
Epoch 6/50
216/216 [==============================] - 74s 345ms/step - loss: 0.9864 - accuracy: 0.8403 - val_loss: 0.9495 - val_accuracy: 0.8800
Epoch 7/50
216/216 [==============================] - 74s 343ms/step - loss: 0.8662 - accuracy: 0.9144 - val_loss: 0.8650 - val_ac

##Model Evaluation

In [21]:
loss, acc = model.evaluate(test_data)
print(f'Test loss:{loss}, Test accuracy:{acc}')

1/1 [==============================] - 16s 16s/step - loss: 0.4882 - accuracy: 1.0000
Test loss:0.4882073402404785, Test accuracy:1.0


##Model Conversion

In [22]:
model.export_model()

In [23]:
!ls exported_model
files.download('exported_model/model.tflite')

checkpoint  metadata.json  model.tflite  summaries


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Retraining parameters

> Indented block



You can further customize how the retraining process runs to adjust training time and potentially increase the retrained model's performance. *These parameters are optional*. Use the `ImageClassifierModelOptions` class and the `HParams` class to set these additional options.

Use the `ImageClassifierModelOptions` class parameters to customize the existing model. It has the following customizable parameter that affects model accuracy:
* `dropout_rate`: The fraction of the input units to drop. Used in dropout layer. Defaults to 0.05.

Use the `HParams` class to customize other parameters related to training and saving the model:

* `learning_rate`: The learning rate to use for gradient descent training. Defaults to 0.001.
* `batch_size`: Batch size for training. Defaults to 2.
* `epochs`: Number of training iterations over the dataset. Defaults to 10.
* `steps_per_epoch`: An optional integer that indicates the number of training steps per epoch. If not set, the training pipeline calculates the default steps per epoch as the training dataset size divided by batch size.
* `shuffle`: True if the dataset is shuffled before training. Defaults to False.
* `do_fine_tuning`: If true, the base module is trained together with the classification layer on top. This defaults to False, which means only the classification layer is trained and pre-trained weights for the base module are frozen.
* `l1_regularizer`: A regularizer that applies a L1 regularization penalty. Defaults to 0.0.
* `l2_regularizer`: A regularizer that applies a L2 regularization penalty. Defaults to 0.0001.
* `label_smoothing`: Amount of label smoothing to apply. See [`tf.keras.losses`](https://www.tensorflow.org/api_docs/python/tf/keras/losses) for more details. Defaults to 0.1.
* `do_data_augmentation`: Whether or not the training dataset is augmented by applying random transformations such as cropping, flipping, etc. See [utils.image_preprocessing](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image) for details. Defaults to True.
* `decay_samples`: Number of training samples used to calculate the decay steps and create the training optimizer. Defaults to 2,560,000.
* `warmup_epochs`: Number of warmup steps for a linear increasing warmup schedule on the learning rate. Used to set up warmup schedule by `model_util.WarmUp`. Defaults to 2.



In [ ]:
hparams=image_classifier.HParams(epochs=15, export_dir="exported_model_2")
options = image_classifier.ImageClassifierOptions(supported_model=spec, hparams=hparams)
model_2 = image_classifier.ImageClassifier.create(
    train_data = train_data,
    validation_data = validation_data,
    options=options,
)

In [ ]:
loss, accuracy = model_2.evaluate(test_data)

In [ ]:
model.export_model()